# Ajout des normales pour les températures et les pressions.

**Auteur :**  Steve Caron  
**Date de création :** 2023/08/29  
**Présentation :** Ce notebook permet de charger des données météorologique depuis une table bigQuery, il calcul les normales pour deux mesures pour chaque jour de l'année pour chaque station. Les valeurs normales, minimums et maximums sont ensuite ajouter à la table de base. Le tout est sauvegardé dans une table BigQuery.


**Prérequis :**  
* Une table BigQuery contenant les données météorologique (TABLE_ID_INPUT)


**Inputs :** 

**Params:**
* TABLE_ID_INPUT : ID de la table BigQuery, doit être dans le même projet.
* Table_ID_OUTPUT : ID de la table BigQuery, doit être dans le même projet.
* LISTE_GROUPBY_INPUT : Le premier element de la liste DOIT ETRE le nom de la colonne avec la date.
* NOM_COLONNE_DATE_AGGREGATION : Nouveaux nom de la colonne date avec laquelle faire l'aggrégation de donnée (exemple: jour_mois, mois, annee)
* FORMAT_DATE_AGGREGATION : Format de la colonne NOM_COLONNE_DATE_AGGREGATION (exemple : dd-MM, MM, yyyy)
* LISTE_COLONNES_A_NORMEES : Liste des colonnes sur lesquelles calculer la norme, doit contenir deux noms de colonne.

In [1]:
from google.cloud import bigquery, storage
from pyspark.sql.functions import col,avg,max,min, date_format
from pyspark.sql.types import DecimalType

In [2]:
ID_TABLE_INPUT = "code-de-source.donnees_code_de_source.donnees_meteo_test"
ID_TABLE_OUTPUT = "code-de-source.donnees_code_de_source.normale_meteo_test_norme"
LISTE_GROUPBY_INPUT = ["date","localite"]
NOM_COLONNE_DATE_AGGREGATION = "jour-mois"
FORMAT_DATE_AGGREGATION = "dd-MM"
LISTE_COLONNES_A_NORMEES = ["precipitation","temperature"]

# Collecte des données depuis une table Bigquery

In [3]:
donnees = spark.read.format('bigquery') \
  .option('table', ID_TABLE_INPUT) \
  .load()

# Ajout de la colonnes date_aggregation

In [4]:
data_jours_mois = donnees.withColumn(NOM_COLONNE_DATE_AGGREGATION,date_format(col(LISTE_GROUPBY_INPUT[0]), FORMAT_DATE_AGGREGATION))

# Ajout des colonnes normes.

Pour chaque colonnes dont on veut calculer la norme, on creer une colonne moyenne,min,max

In [5]:
liste_selection = LISTE_GROUPBY_INPUT[1:] + [NOM_COLONNE_DATE_AGGREGATION] + LISTE_COLONNES_A_NORMEES


table = data_jours_mois.select(liste_selection)
table = table.withColumn(f"{LISTE_COLONNES_A_NORMEES[0]}_normale_min",col(LISTE_COLONNES_A_NORMEES[0]))\
                    .withColumn(f"{LISTE_COLONNES_A_NORMEES[0]}_normale_max",col(LISTE_COLONNES_A_NORMEES[0]))\
                    .withColumnRenamed(LISTE_COLONNES_A_NORMEES[0],f"{LISTE_COLONNES_A_NORMEES[0]}_normale")\
                    .withColumn(f"{LISTE_COLONNES_A_NORMEES[1]}_normale_min",col(LISTE_COLONNES_A_NORMEES[1]))\
                    .withColumn(f"{LISTE_COLONNES_A_NORMEES[1]}_normale_max",col(LISTE_COLONNES_A_NORMEES[1]))\
                    .withColumnRenamed(LISTE_COLONNES_A_NORMEES[1],f"{LISTE_COLONNES_A_NORMEES[1]}_normale")


# Fait une aggrégation de données

In [6]:
# Je vais groupBy sur la liste LISTE_GROUPBY_INPUT, avec la première colonne en moins (celle correspondant à la date) et je la remplace par la colonne "jour-mois"
Liste_groupby = LISTE_GROUPBY_INPUT[1:]+[NOM_COLONNE_DATE_AGGREGATION]

donnees_aggregees = table.groupBy(Liste_groupby) \
    .agg(avg(f"{LISTE_COLONNES_A_NORMEES[0]}_normale").alias(f"{LISTE_COLONNES_A_NORMEES[0]}_normale"), \
         min(f"{LISTE_COLONNES_A_NORMEES[0]}_normale_min").alias(f"{LISTE_COLONNES_A_NORMEES[0]}_normale_min"), \
         max(f"{LISTE_COLONNES_A_NORMEES[0]}_normale_max").alias(f"{LISTE_COLONNES_A_NORMEES[0]}_normale_max"), \
         avg(f"{LISTE_COLONNES_A_NORMEES[1]}_normale").alias(f"{LISTE_COLONNES_A_NORMEES[1]}_normale"),\
         min(f"{LISTE_COLONNES_A_NORMEES[1]}_normale_min").alias(f"{LISTE_COLONNES_A_NORMEES[1]}_normale_min"), \
         max(f"{LISTE_COLONNES_A_NORMEES[1]}_normale_max").alias(f"{LISTE_COLONNES_A_NORMEES[1]}_normale_max"))

# Jointure sur ma table initiale

Je réalise un left join sur les tables data_jours_mois & donnees_aggregees.  
Je renomme les colonnes me permettant de faire la jointure sur la deuxième table pour pouvoir les supprimer facilement.

In [7]:
nouveau_nom_col1 = f"{LISTE_GROUPBY_INPUT[1]}_bis"
nouveau_nom_col_date = f"{NOM_COLONNE_DATE_AGGREGATION}_bis"

donnees_aggregees = donnees_aggregees.withColumnRenamed(LISTE_GROUPBY_INPUT[1],nouveau_nom_col1)\
                                    .withColumnRenamed(NOM_COLONNE_DATE_AGGREGATION,nouveau_nom_col_date)

In [8]:
table_jointe = data_jours_mois.join(donnees_aggregees, (data_jours_mois[LISTE_GROUPBY_INPUT[1]] == donnees_aggregees[nouveau_nom_col1]) &
                                    (data_jours_mois[NOM_COLONNE_DATE_AGGREGATION] == donnees_aggregees[nouveau_nom_col_date]), "left")

# Je supprime les colonnes en trop, resultantes de ma jointure

In [9]:
donnees_finale = table_jointe.drop(col(nouveau_nom_col1)) \
                                .drop(col(nouveau_nom_col_date))

# Envoie de la table normale vers BigQuery

In [10]:
NOM_BUCKET_TEMP = "temp-golden"
LOCATION = "europe-west1"

storage_client = storage.Client()
bucket_temporaire = storage_client.create_bucket(NOM_BUCKET_TEMP, location=LOCATION)

In [11]:
donnees_finale.write \
  .format("bigquery") \
  .option("table",ID_TABLE_OUTPUT)\
  .option("temporaryGcsBucket",NOM_BUCKET_TEMP) \
  .mode("overwrite") \
  .save()

In [12]:
# supprimer le bucket temporaire
bucket_temporaire.delete(force=True)
print("bucket supprimé")

bucket supprimé
